In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import csv 
import tensorflow as tf
import tensorflow_addons as tfa

home = '/kaggle/input/tabular-playground-series-jun-2021'

train_dir = os.path.join(home,'train.csv')
test_dir = os.path.join(home,'test.csv')

#load test data
test_data = pd.read_csv(test_dir, delimiter=',')    

#load train data        
train_data = pd.read_csv(train_dir, delimiter=',')

#concat
all_df = pd.concat([train_data, test_data]).reset_index(drop=True)

# #onehot encroder
# train_label = tf.one_hot(train_label['target'], len(np.unique(train_label['target'])))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Preprocess Data**

In [ ]:
train_data.describe()

In [ ]:
# !pip install dataprep
# from dataprep.eda import plot, plot_correlation, create_report, plot_missing

In [ ]:
# #visualize data to inspect skew, mean, std, and any missing nulls
# plot(train_data.drop(['id'],axis=1))

In [ ]:
# #visualizing skewness as seen above

# train_data.drop(['id','target'], axis=1).skew().plot(kind='bar', figsize=(18,10))

In [ ]:
# cat_features = [col for col in train_data.columns if 'feature_' in col]

In [ ]:
# #apply boxcox to normalize the skewness
# train_data = pd.concat([train_data[cat_features].add(1), train_data[['id','target']]],axis=1)
# test_data = pd.concat([test_data[cat_features].add(1), test_data['id']],axis=1)


# from scipy.stats import boxcox
# for i in train_data.drop(['id','target'], axis=1).columns:
#     train_data[str(i)], lmbda = boxcox(train_data[str(i)], lmbda = None)
#     test_data[str(i)], lmbda = boxcox(test_data[str(i)], lmbda = None)

In [ ]:
# #visualizing skewness after boxcox

# train_data.drop(['id','target'], axis=1).skew().plot(kind='bar', figsize=(18,10))

In [ ]:
X_train = train_data.drop(['id', 'target'],axis=1)
Y_train = train_data['target']

In [ ]:
#onehot encroder
Y_train_label = tf.one_hot([int(x.split('_')[1]) for x in Y_train], 9)

In [ ]:
#split train vs dev set
split = int(0.95*len(X_train))

x_train = X_train[:split]
y_train = Y_train_label[:split]
x_dev = X_train[split:]
y_dev = Y_train_label[split:]

In [ ]:
#DNN
epochs = 50

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=1e-05, patience=8, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.7, patience=2, verbose=0)


#regularization term
regularizer = tf.keras.regularizers.l2(l=0.1)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Embedding(360, 18),
#     tf.keras.layers.GaussianNoise(stddev=0.001, name='gaussian_noise_1'),
#     tf.keras.layers.Conv1D(64, kernel_size=1, activation='relu'),
#     tf.keras.layers.Dropout(0.2, name='dropout_1'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2, name='dropout_1'),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(units=32, activation='selu', kernel_initializer="lecun_normal")),
    tf.keras.layers.Dropout(0.2, name='dropout_2'),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(units=32, activation='relu')),
    tf.keras.layers.Dropout(0.3, name='dropout_3'),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(units=32, activation='elu')),
#     tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer = regularizer),
#     tf.keras.layers.Dense(512, activation ='relu', kernel_regularizer = regularizer),
#     tf.keras.layers.Dense(32, activation ='relu', kernel_regularizer = regularizer),
    tf.keras.layers.Dense(Y_train_label.shape[1], activation='softmax')
])


#hyperparameters for adam 
adam = tf.keras.optimizers.Adam(
    learning_rate=2e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True
)




model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

history = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    verbose=2, 
                    validation_split = 0.95,
                    batch_size = 256
#                     callbacks=[es,plateau]
                   )

In [ ]:
from sklearn.metrics import log_loss
pred_row = model.predict(x_dev)
log_loss(y_dev, pred_row)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()


In [ ]:
test_data.head()

In [ ]:
y_result= model.predict(test_data.iloc[:,1:].values)
print(y_result)

In [ ]:
#submission 

submission_result = pd.DataFrame(y_result,columns=['Class_1','Class_2','Class_3','Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
submission_result['id'] = test_data['id']
submission_result.to_csv('submission.csv', index=False)
submission_result.head()
